In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark import sql
from operator import itemgetter
sc = SparkContext("local", "My Application")
sqlContext = sql.SQLContext(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/05 14:01:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/05 14:01:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/Users/anthonyolund/.local/share/virtualenvs/spark_test-BGiTYayQ/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [25]:
raw_rdd = sc.textFile("data.csv")
raw_rdd.take(1)

['1,I,VXIO456XLBB630221,Nissan,Altima,2003,2002-05-08,Initial sales from TechMotors']

In [24]:
def extract_vin_key_value(x):
    items = itemgetter(2, 1, 3, 5)(x.split(','))
    return (items[0],items[1:])

vin_kv = raw_rdd.map(lambda x: extract_vin_key_value(x))
vin_kv.take(1)

[('VXIO456XLBB630221', ('I', 'Nissan', '2003'))]

In [146]:
def extract_make(x):
    items = itemgetter(2, 3, 5)(x.split(','))
    return (items[0],items[1:])

make = raw_rdd.map(lambda x: extract_make(x))
make.take(1)

[('VXIO456XLBB630221', ('Nissan', '2003'))]

In [147]:
max_make = make.reduceByKey(max)
max_make.take(6)

/users/anthonyolund/documents/dev/apache-spark/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('VXIO456XLBB630221', ('Nissan', '2003')),
 ('INU45KIOOPA343980', ('Mercedes', '2015')),
 ('VOME254OOXW344325', ('Mercedes', '2015')),
 ('EXOA00341AB123456', ('Mercedes', '2016')),
 ('UXIA769ABCC447906', ('Toyota', '2017'))]

In [149]:
rdd_join = vin_kv.join(max_make)
rdd_join.take(10)

[('VOME254OOXW344325', (('I', 'Mercedes', '2015'), ('Mercedes', '2015'))),
 ('VOME254OOXW344325', (('R', '', ''), ('Mercedes', '2015'))),
 ('VOME254OOXW344325', (('A', '', ''), ('Mercedes', '2015'))),
 ('VOME254OOXW344325', (('R', '', ''), ('Mercedes', '2015'))),
 ('VOME254OOXW344325', (('R', '', ''), ('Mercedes', '2015'))),
 ('EXOA00341AB123456', (('I', 'Mercedes', '2016'), ('Mercedes', '2016'))),
 ('EXOA00341AB123456', (('R', '', ''), ('Mercedes', '2016'))),
 ('EXOA00341AB123456', (('A', '', ''), ('Mercedes', '2016'))),
 ('UXIA769ABCC447906', (('I', 'Toyota', '2017'), ('Toyota', '2017'))),
 ('UXIA769ABCC447906', (('R', '', ''), ('Toyota', '2017')))]

In [159]:
enhance_make = rdd_join.map(lambda x: (x[1][0][0], x[1][1][0], x[1][1][1]) )
enhance_make.take(10)

[('I', 'Mercedes', '2015'),
 ('R', 'Mercedes', '2015'),
 ('A', 'Mercedes', '2015'),
 ('R', 'Mercedes', '2015'),
 ('R', 'Mercedes', '2015'),
 ('I', 'Mercedes', '2016'),
 ('R', 'Mercedes', '2016'),
 ('A', 'Mercedes', '2016'),
 ('I', 'Toyota', '2017'),
 ('R', 'Toyota', '2017')]

In [163]:
#The goal of this step is to count the number of records for each make and year combination,
#given the result we derived previously. The output key should be the combination of vehicle
#make and year. The value should be the count of 1.
def extract_make_key_value(x):
    return (x[1:],1)

make_kv = enhance_make.filter(lambda x: x[0] == 'A').map(lambda x: extract_make_key_value(x))
make_kv.take(10)

[(('Mercedes', '2015'), 1),
 (('Mercedes', '2016'), 1),
 (('Nissan', '2003'), 1),
 (('Mercedes', '2015'), 1)]

In [164]:
final_rdd = make_kv.reduceByKey(lambda a,b: a+b)
final_rdd.take(5)

/users/anthonyolund/documents/dev/apache-spark/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[(('Mercedes', '2015'), 2), (('Mercedes', '2016'), 1), (('Nissan', '2003'), 1)]

In [165]:
for element in final_rdd.collect():
    print(element)

(('Mercedes', '2015'), 2)
(('Mercedes', '2016'), 1)
(('Nissan', '2003'), 1)


/users/anthonyolund/documents/dev/apache-spark/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
